Table: Contacts

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| first_name  | varchar |
| last_name   | varchar |
+-------------+---------+
id is the primary key (column with unique values) of this table.
id is a foreign key (reference column) to Calls table.
Each row of this table contains id, first_name, and last_name.
Table: Calls

+-------------+------+
| Column Name | Type |
+-------------+------+
| contact_id  | int  |
| type        | enum |
| duration    | int  |
+-------------+------+
(contact_id, type, duration) is the primary key (column with unique values) of this table.
type is an ENUM (category) type of ('incoming', 'outgoing').
Each row of this table contains information about calls, comprising of contact_id, type, and duration in seconds.
Write a solution to find the three longest incoming and outgoing calls.

Return the result table ordered by type, duration, and first_name in descending order and duration must be formatted as HH:MM:SS.

The result format is in the following example.

 

Example 1:

Input:

Contacts table:

+----+------------+-----------+
| id | first_name | last_name |
+----+------------+-----------+
| 1  | John       | Doe       |
| 2  | Jane       | Smith     |
| 3  | Alice      | Johnson   |
| 4  | Michael    | Brown     |
| 5  | Emily      | Davis     |
+----+------------+-----------+        
Calls table:

+------------+----------+----------+
| contact_id | type     | duration |
+------------+----------+----------+
| 1          | incoming | 120      |
| 1          | outgoing | 180      |
| 2          | incoming | 300      |
| 2          | outgoing | 240      |
| 3          | incoming | 150      |
| 3          | outgoing | 360      |
| 4          | incoming | 420      |
| 4          | outgoing | 200      |
| 5          | incoming | 180      |
| 5          | outgoing | 280      |
+------------+----------+----------+
        
Output:

+-----------+----------+-------------------+
| first_name| type     | duration_formatted|
+-----------+----------+-------------------+
| Alice     | outgoing | 00:06:00          |
| Emily     | outgoing | 00:04:40          |
| Jane      | outgoing | 00:04:00          |
| Michael   | incoming | 00:07:00          |
| Jane      | incoming | 00:05:00          |
| Emily     | incoming | 00:03:00          |
+-----------+----------+-------------------+

In [23]:
import pandas as pd

# Creating the contacts DataFrame
contacts = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'first_name': ['John', 'Jane', 'Alice', 'Michael', 'Emily'],
    'last_name': ['Doe', 'Smith', 'Johnson', 'Brown', 'Davis']
})

print(contacts)


   id first_name last_name
0   1       John       Doe
1   2       Jane     Smith
2   3      Alice   Johnson
3   4    Michael     Brown
4   5      Emily     Davis


In [24]:
import pandas as pd

# Creating the calls DataFrame
calls = pd.DataFrame({
    'contact_id': [1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    'type': ['incoming', 'outgoing', 'incoming', 'outgoing', 'incoming', 'outgoing', 'incoming', 'outgoing', 'incoming', 'outgoing'],
    'duration': [120, 180, 300, 240, 150, 360, 420, 200, 180, 280]
})

print(calls)


   contact_id      type  duration
0           1  incoming       120
1           1  outgoing       180
2           2  incoming       300
3           2  outgoing       240
4           3  incoming       150
5           3  outgoing       360
6           4  incoming       420
7           4  outgoing       200
8           5  incoming       180
9           5  outgoing       280


In [25]:
calls['duration']=pd.to_datetime(calls['duration'],unit='s')
calls

,contact_id,type,duration
0,1,incoming,1970-01-01 00:02:00
1,1,outgoing,1970-01-01 00:03:00
2,2,incoming,1970-01-01 00:05:00
3,2,outgoing,1970-01-01 00:04:00
4,3,incoming,1970-01-01 00:02:30
5,3,outgoing,1970-01-01 00:06:00
6,4,incoming,1970-01-01 00:07:00
7,4,outgoing,1970-01-01 00:03:20
8,5,incoming,1970-01-01 00:03:00
9,5,outgoing,1970-01-01 00:04:40


In [26]:
calls['duration_formatted']=calls['duration'].dt.strftime('%H:%M:%S')
calls

,contact_id,type,duration,duration_formatted
0,1,incoming,1970-01-01 00:02:00,00:02:00
1,1,outgoing,1970-01-01 00:03:00,00:03:00
2,2,incoming,1970-01-01 00:05:00,00:05:00
3,2,outgoing,1970-01-01 00:04:00,00:04:00
4,3,incoming,1970-01-01 00:02:30,00:02:30
5,3,outgoing,1970-01-01 00:06:00,00:06:00
6,4,incoming,1970-01-01 00:07:00,00:07:00
7,4,outgoing,1970-01-01 00:03:20,00:03:20
8,5,incoming,1970-01-01 00:03:00,00:03:00
9,5,outgoing,1970-01-01 00:04:40,00:04:40


In [27]:
calls['rank']=calls.groupby(['type'])['duration_formatted'].rank(method='dense',ascending=False)
calls

,contact_id,type,duration,duration_formatted,rank
0,1,incoming,1970-01-01 00:02:00,00:02:00,5.0
1,1,outgoing,1970-01-01 00:03:00,00:03:00,5.0
2,2,incoming,1970-01-01 00:05:00,00:05:00,2.0
3,2,outgoing,1970-01-01 00:04:00,00:04:00,3.0
4,3,incoming,1970-01-01 00:02:30,00:02:30,4.0
5,3,outgoing,1970-01-01 00:06:00,00:06:00,1.0
6,4,incoming,1970-01-01 00:07:00,00:07:00,1.0
7,4,outgoing,1970-01-01 00:03:20,00:03:20,4.0
8,5,incoming,1970-01-01 00:03:00,00:03:00,3.0
9,5,outgoing,1970-01-01 00:04:40,00:04:40,2.0


In [28]:
joined=calls.merge(contacts,how='inner',left_on='contact_id',right_on='id')
joined

,contact_id,type,duration,duration_formatted,rank,id,first_name,last_name
0,1,incoming,1970-01-01 00:02:00,00:02:00,5.0,1,John,Doe
1,1,outgoing,1970-01-01 00:03:00,00:03:00,5.0,1,John,Doe
2,2,incoming,1970-01-01 00:05:00,00:05:00,2.0,2,Jane,Smith
3,2,outgoing,1970-01-01 00:04:00,00:04:00,3.0,2,Jane,Smith
4,3,incoming,1970-01-01 00:02:30,00:02:30,4.0,3,Alice,Johnson
5,3,outgoing,1970-01-01 00:06:00,00:06:00,1.0,3,Alice,Johnson
6,4,incoming,1970-01-01 00:07:00,00:07:00,1.0,4,Michael,Brown
7,4,outgoing,1970-01-01 00:03:20,00:03:20,4.0,4,Michael,Brown
8,5,incoming,1970-01-01 00:03:00,00:03:00,3.0,5,Emily,Davis
9,5,outgoing,1970-01-01 00:04:40,00:04:40,2.0,5,Emily,Davis


In [29]:
joined[joined['rank']<4][['first_name','type','duration_formatted']].sort_values(by=['type','duration_formatted','first_name'],ascending=False)

,first_name,type,duration_formatted
5,Alice,outgoing,00:06:00
9,Emily,outgoing,00:04:40
3,Jane,outgoing,00:04:00
6,Michael,incoming,00:07:00
2,Jane,incoming,00:05:00
8,Emily,incoming,00:03:00
